In [14]:
import pandas as pd
import numpy as np
X=pd.read_csv('./stream_Xfinal.csv',dtype=object).iloc[:,1:]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83526 entries, 0 to 83525
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   component_name                  83526 non-null  object
 1   component_version               83526 non-null  object
 2   cvm_cpu                         76843 non-null  object
 3   cvm_cpu_qos                     21195 non-null  object
 4   cvm_cpu_type                    74843 non-null  object
 5   cvm_gpu_type                    999 non-null    object
 6   cvm_memory                      76815 non-null  object
 7   cvm_os_type                     76837 non-null  object
 8   cvm_version                     69524 non-null  object
 9   host_configured_clock_speed     8189 non-null   object
 10  host_cpu_isolation              56 non-null     object
 11  host_cpu_isolation:dpdk         2586 non-null   object
 12  host_cpu_isolation:host         2586 non-null 

In [15]:
# 舍弃阈值
threshold=len(X)*0.5
delete_column=[]
for column in X.columns:
    if(X[column].count()<threshold):
        delete_column.append(column)
X.drop(columns=delete_column,inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83526 entries, 0 to 83525
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   component_name     83526 non-null  object
 1   component_version  83526 non-null  object
 2   cvm_cpu            76843 non-null  object
 3   cvm_cpu_type       74843 non-null  object
 4   cvm_memory         76815 non-null  object
 5   cvm_os_type        76837 non-null  object
 6   cvm_version        69524 non-null  object
 7   platform           83214 non-null  object
 8   test_name          83526 non-null  object
 9   tool_name          83526 non-null  object
 10  tool_version       83526 non-null  object
 11  results_key        83526 non-null  object
dtypes: object(12)
memory usage: 7.6+ MB


In [16]:
#相同配置取最后一项，这是由于数据有时序性，且发现数据集中较早测的有严重偏差
X=X.drop_duplicates(keep='last')
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18435 entries, 1 to 83525
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   component_name     18435 non-null  object
 1   component_version  18435 non-null  object
 2   cvm_cpu            18099 non-null  object
 3   cvm_cpu_type       16511 non-null  object
 4   cvm_memory         18085 non-null  object
 5   cvm_os_type        18050 non-null  object
 6   cvm_version        15512 non-null  object
 7   platform           18338 non-null  object
 8   test_name          18435 non-null  object
 9   tool_name          18435 non-null  object
 10  tool_version       18435 non-null  object
 11  results_key        18435 non-null  object
dtypes: object(12)
memory usage: 1.8+ MB


In [17]:
X=X[["cvm_cpu","results_key","cvm_memory","cvm_cpu_type","cvm_os_type"]]
X["cvm_cpu"]=X["cvm_cpu"].fillna(-1)
X["cvm_memory"]=X["cvm_memory"].fillna(-1)

In [18]:
for i in range(len(X)):
    index=X.index[i]
    try:
        X["cvm_memory"][index]=X["cvm_memory"][index].split(" ")[0]
    except:
        continue
X.dropna(inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16466 entries, 1 to 83525
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cvm_cpu       16466 non-null  object
 1   results_key   16466 non-null  object
 2   cvm_memory    16466 non-null  object
 3   cvm_cpu_type  16466 non-null  object
 4   cvm_os_type   16466 non-null  object
dtypes: object(5)
memory usage: 771.8+ KB


In [19]:
X[["cvm_cpu","cvm_memory"]]=X[["cvm_cpu","cvm_memory"]].astype(int)
X_encode=pd.get_dummies(X)
X_encode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16466 entries, 1 to 83525
Columns: 202 entries, cvm_cpu to cvm_os_type_tlinux
dtypes: int32(2), uint8(200)
memory usage: 3.4 MB


In [20]:
y_final=pd.read_csv('./stream_yfinal.csv').iloc[:,1:]
y_final=y_final.iloc[X.index]
y_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16466 entries, 1 to 83525
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   #stream_add         1513 non-null   float64
 1   #stream_copy        15366 non-null  float64
 2   #stream_copy_m      2 non-null      float64
 3   #stream_copy_numa   0 non-null      float64
 4   #stream_ht1_copy    982 non-null    float64
 5   #stream_ht1_triad   982 non-null    float64
 6   #stream_ht2_copy    982 non-null    float64
 7   #stream_ht2_triad   982 non-null    float64
 8   #stream_scale       1513 non-null   float64
 9   #stream_triad       15366 non-null  float64
 10  #stream_triad_m     2 non-null      float64
 11  #stream_triad_numa  0 non-null      float64
 12  stream_add          13662 non-null  float64
 13  stream_add_m        2 non-null      float64
 14  stream_add_numa     0 non-null      float64
 15  stream_ht1_add      982 non-null    float64
 16  stre

In [21]:
threshold = len(y_final) * 0.5
delete_column = y_final.apply(lambda x: x.count() < threshold, axis=0)
y_final.drop(columns=y_final.columns[delete_column], inplace=True)
delete_column

#stream_add            True
#stream_copy          False
#stream_copy_m         True
#stream_copy_numa      True
#stream_ht1_copy       True
#stream_ht1_triad      True
#stream_ht2_copy       True
#stream_ht2_triad      True
#stream_scale          True
#stream_triad         False
#stream_triad_m        True
#stream_triad_numa     True
stream_add            False
stream_add_m           True
stream_add_numa        True
stream_ht1_add         True
stream_ht1_copy        True
stream_ht1_scale       True
stream_ht1_triad       True
stream_ht2_add         True
stream_ht2_copy        True
stream_ht2_scale       True
stream_ht2_triad       True
stream_scale          False
stream_scale_m         True
stream_scale_numa      True
dtype: bool

In [22]:
for column in y_final:
    y_final[column].fillna(y_final[column].mean(),inplace=True)
y_final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16466 entries, 1 to 83525
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   #stream_copy   16466 non-null  float64
 1   #stream_triad  16466 non-null  float64
 2   stream_add     16466 non-null  float64
 3   stream_scale   16466 non-null  float64
dtypes: float64(4)
memory usage: 643.2 KB


In [23]:
from sklearn.model_selection import train_test_split,cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X_encode, y_final, test_size=0.2, random_state=1)

In [24]:
# 导入随机森林回归模型和网格搜索模块
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# 定义要搜索的参数范围
param_grid = {'max_depth': range(10, 50, 5)}

# 创建随机森林回归模型
rfg = RandomForestRegressor(random_state=42)

# 定义网格搜索模型
grid_search = GridSearchCV(rfg, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')

# 在训练集上拟合模型
grid_search.fit(X_train, y_train)

# 输出最佳超参数
best_params = grid_search.best_params_
print("best_params:", best_params)

# 获取最佳模型
best_model = grid_search.best_estimator_

# 使用最佳模型进行预测
y_pred = best_model.predict(X_test)

# 输出最佳深度
print("Best max_depth value: {}".format(best_model.max_depth))

best_params: {'max_depth': 45}
Best max_depth value: 45


In [25]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
r2=r2_score(y_test,y_pred)
print("r2_score",r2)

r2_score 0.9420640092049313


In [26]:
# 定义多个max_depth值
max_depths = [15,25,35,45]

# 创建随机森林回归模型，并拟合数据
models = []
for max_depth in max_depths:
    model = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=42,n_jobs=-1)
    model.fit(X_train, y_train)
    models.append(model)

# 计算每个模型的R2和RMSE评分
r2_scores = []
rmses = []
maes = []
for model in models:
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae=mean_absolute_error(y_test,y_pred)
    r2_scores.append(r2)
    rmses.append(rmse)
    maes.append(mae)
    
# 输出结果
for i in range(len(max_depths)):
    print("Max_depth = {}: R2 score = {:.3f}, RMSE = {:.3f}, MAE = {:.3f}".format(max_depths[i], r2_scores[i], rmses[i],maes[i]))

Max_depth = 15: R2 score = 0.915, RMSE = 13703.344, MAE = 8015.478
Max_depth = 25: R2 score = 0.935, RMSE = 12000.532, MAE = 5571.979
Max_depth = 35: R2 score = 0.940, RMSE = 11455.912, MAE = 4549.901
Max_depth = 45: R2 score = 0.942, RMSE = 11308.502, MAE = 4260.918
